In [ ]:
import os
import sys
import pandas as pd
import datetime as dt

import panel as pn
from panel.interact import interact
import altair as alt
alt.data_transformers.enable('data_server')
alt.renderers.enable('mimetype')

src_path = os.path.abspath(os.path.join(".."))
if src_path not in sys.path:
    sys.path.append(src_path)

from src.data import navigation_events

%load_ext autoreload
%autoreload 2

In [ ]:
raw_events_df = navigation_events.load_events_data(os.path.join(os.pardir, "data", "navigation_events.csv"))

In [ ]:
clean_events_df = navigation_events.preprocess_events_data(raw_events_df)

In [ ]:
membership_plot = alt.Chart(clean_events_df).mark_bar().encode(
    y = "membership_role",
    x = "count()",
    color = "membership_role",
    tooltip = "membership_role"
).properties(
    height = 600,
    width = 400
)
membership_plot

In [ ]:
session_df = navigation_events.group_by_session_data(clean_events_df)

In [ ]:
alt.Chart(session_df).mark_point(clip=True).encode(
    x = "monthdate(session_start):T",
    y = alt.Y("sum(discussion_interactions)", scale = alt.Scale(domain = [0,200])), #scale = alt.Scale(domain = [0,5000],
    # color = "student",
    # tooltip = "student"
).properties(
    width = 1200,
)

In [ ]:
start_date =  pd.Timestamp(2033, 2,25)
end_date =  pd.Timestamp(2033, 3,25)

start_date =  dt.date(2033, 2,25)
end_date =  dt.date(2033, 3,25)

user_summary_df = navigation_events.timeframe_event_summary(session_df, start_date, end_date)

In [ ]:
user_summary_df.head()

In [ ]:
perc_columns = user_summary_df.columns[user_summary_df.columns.str.contains("__perc")]

def bar_plot(x_col, df = user_summary_df):
    # Plot the sorted and filtered data frame
    # chart = alt.Chart(df).mark_point().encode(
    #     x='Horsepower',
    #     y=y_col)
    # return chart
    plot = alt.Chart(df).mark_bar().encode(
        x = x_col,
        y = alt.Y("student", sort = "y"),
        color  ="student"
    ).properties(
        height = 500,
        width = 500
    )

    return plot

# Add dropdown menus
interact(bar_plot, x_col=perc_columns).embed(max_opts=100)

In [ ]:
students = sorted(user_summary_df['student'].unique())
dropdown = alt.binding_select(options=students)

# pn.extension('vega')

select_student = alt.selection_single(
    fields=['student'],
    bind=dropdown)

alt.Chart(user_summary_df).mark_bar().encode(
    x = alt.condition(select_student, "avg_percentile", alt.value(0.1)),
    y = alt.Y("student", sort = "-x"),
    opacity=alt.condition(select_student, alt.value(0.9), alt.value(0.1))
).properties(
    height = 500,
    width = 500
).add_selection(select_student)